FinBERT를 활용해서 뉴스 기사의 감성 점수를 계산하는 예시를 보여준다. 

In [47]:
import numpy as np 
import pandas as pd 
from transformers import * # pip install transformers==4.3.3
from googletrans import Translator # pip install googletrans==4.0.0-rc1 
import torch # pip install torch==1.7.1 
import torch.nn as nn

In [7]:
### define FinBERT model for sentiment classification ### 
finbert = BertForSequenceClassification.from_pretrained('ProsusAI/finbert',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

In [66]:
### text scraped from coinness ### 
kor_text = '''
ETH 단기 하락, 3,900 달러선 일시 반납
코인니스 마켓 모니터링에 따르면, ETH가 단기 하락 흐름을 나타내며 3,900 달러선을 일시 반납했다. 바이낸스 USDT 마켓 기준 ETH는 현재 4.15% 내린 3,901.05 달러에 거래되고 있다.
'''

### translate text from Korean to English ### 
translator = Translator() 
eng_text = translator.translate(kor_text).text 

print(eng_text)
print()

### tokenize input text for model processing ### 
encoded_dict = tokenizer.encode_plus(
        text = eng_text, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = True, 
        return_attention_mask = True, # constructing attention_masks 
        max_length = 512) 

input_ids = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask'] 
token_type_id = encoded_dict['token_type_ids'] 

input_ids = torch.tensor(input_ids)
input_ids = torch.reshape(input_ids,(1,-1)) 

attention_mask = torch.tensor(attention_mask)
attention_mask = torch.reshape(attention_mask, (1,-1)) 

token_type_id = torch.tensor(token_type_id) 
token_type_id = torch.reshape(token_type_id, (1,-1)) 


### calculate sentiment score using FinBERT ### 
finbert.eval() 
with torch.no_grad(): 
    output = finbert(input_ids, attention_mask, token_type_id)

criterion = nn.Softmax(dim=1) 
scores = criterion(output['logits']) 

print("Calculations...")
print("Positive sentiment: {:.3f}%".format(scores[0][0] * 100))
print("Negative sentiment: {:.3f}%".format(scores[0][1] * 100))
print("Neutral sentiment: {:.3f}%".format(scores[0][2] * 100)) 

ETH Short-Term Drop, Temporary Refund to $3,900
According to Coinness Market Monitoring, ETH temporarily returned $3,900, indicating a short-term downward trend. On the Binance USDT market, ETH is currently trading at $3,901.05, down 4.15%.

Calculations...
Positive sentiment: 0.783%
Negative sentiment: 97.541%
Neutral sentiment: 1.676%
